In [2]:
import pandas as pd
import numpy as np


In [3]:
def cambiar_release(cadena_fecha):
    """
    Esta funcion extrae unicamente el año del release_date
    """
    if cadena_fecha==None:
        return None
    else:
        return cadena_fecha.split("-")[0]



# ETL DE DF_REDUCIDO

In [4]:
def evaluar_presencia_user(user_id):
    if user_id!=None:
        return True
    else:
        return False

In [5]:
def valida_anio(anio1,anio2):
    # Validamos que el año de 2 fechas sean consecuentes para evaluar en los filtros de la funcion counterreviews
    anio1 = int(anio1)
    anio2 = int(anio2)
    if anio1>anio2:
        return True
    elif anio1>2023 or anio2>2023 or anio1<1950 or anio2<1950:
        return True
    else:
        return False

In [11]:
df_games = pd.read_json("clean_games.json")
df_reviews = pd.read_parquet("clean_reviews.parquet")
df_items = pd.read_parquet("australian_users_items.parquet")
df_genres = pd.read_json("clean_genre.json")
df_lista_genre = pd.read_json("dataframe_genre.json")
df_playtime = pd.read_parquet("parquet_generos.parquet")

df_reviews_copia1 = df_reviews.copy()

# Ubicamos las filas donde  no se tenga informacion de posted
indices_2 = df_reviews_copia1["posted"][df_reviews_copia1["posted"].apply(lambda x: x==None)].index

# Para la consigna se trabaja unicamente los datos donde se conozca los años
df_reviews_copia1.drop(indices_2,inplace=True,axis=0)
df_reviews_copia1.reset_index(drop = True, inplace=True)



# Aqui estamos pasando a formato fecha
df_reviews_copia1["posted"] = pd.to_datetime(df_reviews["posted"])



# Creamos un nuevo dataframe DF_REVIEWS_COPIA1


In [6]:


# Reducimos el dataframe solo con las variables que necesitamos

df_reducido = df_games.loc[:,["release_date","price","developer"]]
df_reducido["release_date"] = df_reducido["release_date"].apply(lambda x: cambiar_release(x))

# CREAMOS OTRO DATAFRAME EN BASE A GAMES

# Creamos un dataframe DF_COPIA
df_reviews_copia2 = df_reviews.copy()
df_reviews_copia2["posted"] = df_reviews_copia2["posted"].apply(lambda x: cambiar_release(x))


## Probando la funcion 1

In [7]:
def user_data(user_id):
    ''' Esta funcion retornara la cantidad de dinero que el usuario gasto en los juegos de steam, además devolvera un porcentaje de recomendaciones
      en base a la cantidad de items(juegos) que posea

      parameters:

        user_id : id del usuario
    '''
    if evaluar_presencia_user(user_id): 
        user_id=str(user_id)
        
        # cantidad_items = df_items[df_items["user_id"]==user_id].shape[0]
        data = df_items[df_items["user_id"]==user_id]
        cantidad_juegos = data.shape[0]
        mascara = data["item_id"].isin(df_games["id"])

        # Aca observamos la cantidad de juegos del dataframe df_games que coinciden con los registros de data_items
        juegos =data[mascara]   

        # filas en las que se tiene la concidencia por id del item
        indices = np.where(df_games['id'].isin(juegos["item_id"]))[0]

        precio_juegos =df_games.loc[indices,"price"]
        gasto = sum(precio_juegos)

        # Con esto accederemos a los reviews del usuario
        review_usuario = df_reviews[df_reviews["user_id"]== user_id]

        # Uso  el dataframe juegoss["item_id"] para filtrar los juegos que tienen recommend

        mascara3 = review_usuario["item_id"].isin(juegos["item_id"])
        commend_filas =review_usuario[mascara3]

        # Se vuelve a hacer este filtrado pues se requiere certificar si realmente los juegos que coinciden entre los que el usuario tiene son la misma cantidad
        # Que los que haya hecho review

        suma_recommends = 0
        for i in range(commend_filas.shape[0]):
          if commend_filas.iloc[i,4]!=None:
            suma_recommends+=commend_filas.iloc[i,4]
        # Sumamos las recomendaciones, sabiendo los elementos True y False son tomados como elementos binario en la suma
        try:
           porcentaje = (suma_recommends/cantidad_juegos)*100
        except ZeroDivisionError:
          porcentaje = 0

        return f"""La cantidad de dinero gastado por el usuario {user_id} es: {round(gasto,3)}
El porcentaje de recomendación en base a su cantidad de items de: {cantidad_juegos} es {porcentaje}%"""
        

    else:
       return "Este usuario no existe"

In [8]:
data = user_data("76561197970982479")
print(data)

La cantidad de dinero gastado por el usuario 76561197970982479 es: 0
El porcentaje de recomendación en base a su cantidad de items de: 277 es 0.0%


# Ahora probaremos la segunda funcion

In [9]:
def countreviews(fecha1,fecha2):
    """Esta funcion retornara la cantidad de usuarios y el porcentaje de recomendaciones entre 2 fechas específicas.

       parameters:
       fecha1: fecha inicial(start) con formato YYYY-MM-DD
       fecha2: fecha final(end) con formato YYYY-MM-DD
    """
    #Validamos las fechas que se ingresan
    lista_fecha1=fecha1.split("-")
    lista_fecha2=fecha2.split("-")
    if valida_anio(lista_fecha1[0],lista_fecha2[0]):
        return "Fechas Incorrectas"
    elif (int(lista_fecha1[0])==int(lista_fecha2[0])) and (int(lista_fecha1[1])>int(lista_fecha2[1])):
        return "Fechas Incorrectas"
    elif (int(lista_fecha1[0])==int(lista_fecha2[0])) and (int(lista_fecha1[1])==int(lista_fecha2[1])) and (int(lista_fecha1[2])>int(lista_fecha2[2])):
        return "Fechas Incorrectas"
    else:
        filas = df_reviews_copia1["posted"][(df_reviews_copia1["posted"]>fecha1) & (df_reviews_copia1["posted"]<fecha2)].index
        df_rango_fechas = df_reviews_copia1.loc[filas,:]
        cantidad_personas = df_rango_fechas.shape[0]
        recommends = sum(df_rango_fechas["recommend"][df_rango_fechas["recommend"]==True])
        try:
            porcentaje = (recommends/cantidad_personas)*100
        except ValueError:
            return f"""La cantidad de usuarios entre las fechas {fecha1} y {fecha2} es: {cantidad_personas},
        ademas el porcentaje de recomendacion es 0%"""
        except ZeroDivisionError:
            return f"""La cantidad de usuarios entre las fechas {fecha1} y {fecha2} es: {cantidad_personas},
        ademas el porcentaje de recomendacion es 0%"""
        
        else:
            return f"""La cantidad de usuarios entre las fechas {fecha1} y {fecha2} es: {cantidad_personas},
ademas el porcentaje de recomendacion es {porcentaje}%"""

In [10]:
fechas = countreviews("2013-11-23","2014-01-20")
print(fechas)

La cantidad de usuarios entre las fechas 2013-11-23 y 2014-01-20 es: 4443,
ademas el porcentaje de recomendacion es 89.46657663740716%


## Ahora probaremos la CUARTA funcion

In [67]:
def userforgenre(genero):
    genero="genre_"+genero
    if genero in df_playtime.columns:
    
        resultado = df_playtime
        # Procedemos a  sacar el top de usuarios que hayan jugado mas por genero
        resultado.sort_values(by="playtime_forever",ascending = False,inplace = True)
        users = resultado.iloc[0:5,:].loc[:,["user_id","user_url","playtime_forever"]]
        dicciona = users.to_dict(orient="records") 
        
        return dicciona
        return "Lo siento, ningun juego tiene este género"

In [68]:
df_super_seleccion = userforgenre("Education")
df_super_seleccion

[{'user_id': 'wolop',
  'user_url': 'http://steamcommunity.com/id/wolop',
  'playtime_forever': 642773},
 {'user_id': 'Evilutional',
  'user_url': 'http://steamcommunity.com/id/Evilutional',
  'playtime_forever': 635295},
 {'user_id': '76561198039832932',
  'user_url': 'http://steamcommunity.com/profiles/76561198039832932',
  'playtime_forever': 613411},
 {'user_id': 'tsunamitad',
  'user_url': 'http://steamcommunity.com/id/tsunamitad',
  'playtime_forever': 600068},
 {'user_id': 'jimmynoe',
  'user_url': 'http://steamcommunity.com/id/jimmynoe',
  'playtime_forever': 551719}]

In [58]:
df_super_seleccion["user_id"].values

array(['wolop', 'Evilutional', '76561198039832932', 'tsunamitad',
       'jimmynoe'], dtype=object)

In [59]:
df_super_seleccion["user_url"].values

array(['http://steamcommunity.com/id/wolop',
       'http://steamcommunity.com/id/Evilutional',
       'http://steamcommunity.com/profiles/76561198039832932',
       'http://steamcommunity.com/id/tsunamitad',
       'http://steamcommunity.com/id/jimmynoe'], dtype=object)

In [61]:
df_super_seleccion["playtime_forever"].values

array([642773, 635295, 613411, 600068, 551719], dtype=int64)

## Veamos la TERCERA funcion

In [51]:
def genre(genero):
    """
    Esta funcion devuelve el puesto en el que se encuentra dicho genero
    """
    
    if genero in df_lista_genre["genre"].values:
        filtro = df_lista_genre["playtime_forever(minutes)"][df_lista_genre["genre"]==genero]
        puesto = filtro.index[0]
        horas = filtro.values[0]
        return f"El puesto de {genero} es {puesto}, con {horas} horas"
        
    else:
        return "Este genero no existe"

In [47]:
filtro = df_lista_genre["playtime_forever(minutes)"][df_lista_genre["genre"]=="Education"]

In [50]:
puesto = filtro.index[0]
horas = filtro.values[0]
puesto,horas

(19, 415556)

In [52]:
listado = genre("Education")
listado

'El puesto de Education es 19, con 415556 horas'

## Ahora haremos la funcion 5

In [ ]:
def developer(desarrollador):
    """
    Esta funcion extrae el porcentaje de juegos que han sido Free(sin costo) por desarrollador y por año
    """
    df_desarrollador = df_reducido[df_reducido["developer"]==desarrollador]
    fechas = df_desarrollador["release_date"].unique()
    
    # Creamos un diccionario con los pares años, y precios
    diccionario_anios = {}
    diccionario_tamanio = {}
    for i in fechas:
        df_free = df_desarrollador[df_desarrollador["release_date"]==i]
        diccionario_anios[i]=len(df_free[df_free["price"]==0])
        diccionario_tamanio[i]=df_free.shape[0]

    # Se extrae la cantidad de items en total, y aquellos que no son free

    diccionario_cantidades=  {}

    for i in diccionario_anios:
        try:
            diccionario_cantidades[i]= (diccionario_anios[i]/diccionario_tamanio[i])*100
        except ZeroDivisionError:
            diccionario_cantidades[i]=0
            
    # Se muestra el porcentaje de juegos free del desarrollador por año
    df_mostrar = pd.DataFrame({'años': diccionario_cantidades.keys() , "Porcentaje Free(%)" : diccionario_cantidades.values()})
    return df_mostrar

In [ ]:
prueba = developer("Secret Level SRL")
prueba

## Veamos la ultima funcion

In [11]:
def sentiment_analysis(anio):
    anio = str(anio)
    if anio <= "2015":
        seleccionado = df_reviews_copia2[df_reviews_copia2["posted"] == anio]
        resultado = seleccionado["review"].value_counts()
        final = pd.DataFrame(resultado)
        final.index = ["Neutro", "Positivo", "Negativo"]
        return final
    else:
        return "No hay data de ese año"


In [12]:
def trans_review(cadena):
    if type(cadena)!=str:
        return str(cadena)
    else:
        return cadena

In [13]:
# PASO PREVIO

df_reviews_copia2 = df_reviews_copia2.dropna(subset=["posted"])
df_reviews_copia2.reset_index(drop=True,inplace=True)
df_reviews_copia2["posted"] = df_reviews_copia2["posted"].apply(lambda x: trans_review(x))

In [ ]:
df_reviews_copia2["posted"] = df_reviews_copia2["posted"].apply(lambda x: trans_review(x))

In [18]:
df_reviews_copia2["posted"].astype(str)

0        2011
1        2011
2        2011
3        2014
4        2013
         ... 
49181    2015
49182    2015
49183    2015
49184    2015
49185    2015
Name: posted, Length: 49186, dtype: object

In [24]:
df_reviews_copia2[df_reviews_copia2["posted"].apply(lambda x: isinstance(x,str))]

,user_id,user_url,posted,item_id,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011,1250,True,1
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011,22200,True,1
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011,43110,True,1
3,js41637,http://steamcommunity.com/id/js41637,2014,251610,True,1
4,js41637,http://steamcommunity.com/id/js41637,2013,227300,True,1
...,...,...,...,...,...,...
49181,wayfeng,http://steamcommunity.com/id/wayfeng,2015,730,True,1
49182,76561198251004808,http://steamcommunity.com/profiles/76561198251...,2015,253980,True,1
49183,72947282842,http://steamcommunity.com/id/72947282842,2015,730,True,0
49184,ApxLGhost,http://steamcommunity.com/id/ApxLGhost,2015,730,True,2


In [14]:
df_reviews_copia2["posted"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 49186 entries, 0 to 49185
Series name: posted
Non-Null Count  Dtype 
--------------  ----- 
49186 non-null  object
dtypes: object(1)
memory usage: 384.4+ KB


In [15]:
seleccionado = df_reviews_copia2[df_reviews_copia2["posted"] == "2011"]

In [16]:
seleccionado

,user_id,user_url,posted,item_id,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011,1250,True,1
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011,22200,True,1
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011,43110,True,1
267,2xDelorean,http://steamcommunity.com/id/2xDelorean,2011,440,True,2
400,GamerFag,http://steamcommunity.com/id/GamerFag,2011,70,True,1
...,...,...,...,...,...,...
39274,4i20swagyolo69,http://steamcommunity.com/id/4i20swagyolo69,2011,4000,True,1
39426,euphoria72,http://steamcommunity.com/id/euphoria72,2011,570,True,2
39444,76561198049927258,http://steamcommunity.com/profiles/76561198049...,2011,105600,True,1
39473,RyuichiRyuu,http://steamcommunity.com/id/RyuichiRyuu,2011,440,True,1


In [17]:
resultado = seleccionado["review"].value_counts()
resultado

review
1    417
2    110
0      5
Name: count, dtype: int64

In [ ]:
seleccionado = df_reviews_copia2[df_reviews_copia2["posted"] == anio]
resultado = seleccionado["review"].value_counts()
final = pd.DataFrame(resultado)

In [36]:
def sentiment_analysis(anio):
   
    if anio <= 2015:
        anio = str(anio)
        seleccionado = df_reviews_copia2[df_reviews_copia2["posted"] == anio]
        resultado = seleccionado["review"].value_counts()
        resultado = resultado.to_dict()
        return dict(zip(["Neutro","Positivo","Negativo"],resultado.values()))
    else:
        return "No hay data de ese año"


In [29]:
seleccionado = df_reviews_copia2[df_reviews_copia2["posted"] == "2011"]
resultado = seleccionado["review"].value_counts()

In [33]:
resultado.to_dict()

{1: 417, 2: 110, 0: 5}

In [37]:
sentiment_analysis(2011)

{'Neutro': 417, 'Positivo': 110, 'Negativo': 5}